In [2]:
pip install tensorflow numpy scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

FordA 데이터 로드

In [7]:
def load_ucr(dataset='FordA'):
    base_url = f"https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"
    train_data = np.loadtxt(base_url + f"{dataset}_TRAIN.tsv")
    test_data = np.loadtxt(base_url + f"{dataset}_TEST.tsv")

    X_train, y_train = train_data[:, 1:], train_data[:, 0]
    X_test, y_test = test_data[:, 1:], test_data[:, 0]

    # 클래스 라벨이 -1, 1로 되어 있으니 0, 1로 바꿔줌
    y_train = (y_train == 1).astype(int)
    y_test = (y_test == 1).astype(int)

    return X_train, y_train, X_test, y_test

데이터 로드

In [11]:
X_train, y_train, X_test, y_test = load_ucr()

데이터 dimension 맞추기

In [14]:
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

MODEL DEFINITION

In [17]:
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 이진 분류
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\rlfrn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


학습

In [20]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.5297 - loss: 0.6938 - val_accuracy: 0.5257 - val_loss: 0.6867
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.5484 - loss: 0.6865 - val_accuracy: 0.5603 - val_loss: 0.6710
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.5397 - loss: 0.6670 - val_accuracy: 0.5895 - val_loss: 0.6379
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.6263 - loss: 0.6274 - val_accuracy: 0.6755 - val_loss: 0.6068
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.6911 - loss: 0.5723 - val_accuracy: 0.6574 - val_loss: 0.5668
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.7087 - loss: 0.5496 - val_accuracy: 0.6879 - val_loss: 0.5597
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.7107 - loss: 0.5282 - val_accuracy: 0.6976 - val_loss: 0.5455
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.7192 - loss: 0.5015 - val_accuracy: 0.7046 - v

평가

In [23]:
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ 테스트 정확도: {acc * 100:.2f}%")

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7736 - loss: 0.4073

✅ 테스트 정확도: 76.52%


분류 결과

In [26]:
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
              precision    recall  f1-score   support

           0       0.85      0.66      0.74       681
           1       0.71      0.87      0.78       639

    accuracy                           0.77      1320
   macro avg       0.78      0.77      0.76      1320
weighted avg       0.78      0.77      0.76      1320

